In [37]:
%matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from preprocessor import read_mat_trials_ch as matread

from keras.models import Sequential
from keras.layers import Dense, Dropout, SimpleRNN
from keras.optimizers import SGD, RMSprop, Nadam
from keras.constraints import maxnorm

Using matplotlib backend: Qt5Agg


In [38]:
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

<IPython.core.display.Javascript object>

In [39]:
def load_data(sub, sf, n, win, num_samples):
    
    # Read files, preprocess and get data for 'B' and sf ('f1'/'f2')
    data_s = matread(sf, sub)
    data_b = matread('B', sub)
    
    train_s = data_s[:num_samples, :, :]
    train_b = data_b[:num_samples, :, :]
    test_s = data_s[num_samples:, :, :]
    test_b = data_b[num_samples:, :, :]
    
    gns, nT, nC = data_s.shape
    ws = win + 1; we = nT - ws
    
    temp_s, temp_b = [], []
    for i in range(num_samples):
        for j in range(n):
            rands = random.randint(ws, we)
            randb = random.randint(ws, we)
            temp_s.append(train_s[i][rands:rands+win, :])
            temp_b.append(train_b[i][randb:randb+win, :])

    x_train = np.asarray(temp_s)
    x_train = np.append(x_train, np.asarray(temp_b), axis=0)
    y_train = np.zeros([n * num_samples * 2, 2])
    y_train[:n * num_samples, 0] = 1
    y_train[n * num_samples:, 1] = 1

    x_train, y_train = shuffle(x_train, y_train, random_state=0)
    
    temp_s, temp_b = [], []
    for i in range(gns - num_samples):
        for j in range(n):
            rands = random.randint(ws, we)
            randb = random.randint(ws, we)
            temp_s.append(test_s[i][rands:rands+win, :])
            temp_b.append(test_b[i][randb:randb+win, :])

    x_test = np.asarray(temp_s)
    x_test = np.append(x_test, np.asarray(temp_b), axis=0)
    num_test_samples = x_test.shape[0]
    y_test = np.zeros([num_test_samples, 2])
    y_test[:n * num_samples, 0] = 1
    y_test[n * num_samples:, 1] = 1
       
    return x_train, x_test, y_train, y_test

In [40]:
def model_fit(model, batch_size, num_epochs, val_split):
    hist = []
    hist.append(model.fit(x_train, y_train, batch_size=batch_size, epochs=num_epochs, verbose=2, validation_split=val_split))
    
    acc1 = hist[0].history['val_acc']
    acc0 = hist[0].history['acc']
    return [acc0[-1:], acc1[-1:]], acc0, acc1

In [90]:
def create_model():
    model = Sequential()
       
    model.add(Dense(8, batch_input_shape=(None, x_train.shape[1], x_train.shape[2]),
                activation='linear'))
    model.add(Dropout(0.1))
    model.add(SimpleRNN(16, activation='tanh', kernel_initializer='random_uniform',
                    recurrent_initializer='random_uniform', bias_initializer='zeros'))
    model.add(Dense(2, activation='sigmoid'))
#     opt = RMSprop(lr=0.001)
    opt = Nadam()
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [97]:
%%notify
# Parameters
sub, sf = 1, 'f1'
n = 25
win = 400
num_samples = 12
batch_size = 40
val_split = 0.25
num_epochs = 400

model = None
model = create_model()

res = []
for t in range(10):
    x_train, x_test, y_train, y_test = load_data(sub, sf, n, win, num_samples)
    amat, a0, a1 = model_fit(model, batch_size, num_epochs, val_split)
    res.append(amat)

Train on 450 samples, validate on 150 samples
Epoch 1/400
 - 6s - loss: 0.6934 - acc: 0.4844 - val_loss: 0.6941 - val_acc: 0.5267
Epoch 2/400
 - 0s - loss: 0.6922 - acc: 0.5178 - val_loss: 0.6955 - val_acc: 0.5333
Epoch 3/400
 - 0s - loss: 0.6919 - acc: 0.4956 - val_loss: 0.6957 - val_acc: 0.5333
Epoch 4/400
 - 0s - loss: 0.6913 - acc: 0.5067 - val_loss: 0.6966 - val_acc: 0.5467
Epoch 5/400
 - 0s - loss: 0.6914 - acc: 0.5111 - val_loss: 0.6976 - val_acc: 0.4933
Epoch 6/400
 - 0s - loss: 0.6912 - acc: 0.5044 - val_loss: 0.6995 - val_acc: 0.4667
Epoch 7/400
 - 0s - loss: 0.6912 - acc: 0.5156 - val_loss: 0.6990 - val_acc: 0.4867
Epoch 8/400
 - 0s - loss: 0.6917 - acc: 0.5133 - val_loss: 0.6992 - val_acc: 0.4867
Epoch 9/400
 - 0s - loss: 0.6920 - acc: 0.4956 - val_loss: 0.6989 - val_acc: 0.4733
Epoch 10/400
 - 0s - loss: 0.6910 - acc: 0.5156 - val_loss: 0.6992 - val_acc: 0.4667
Epoch 11/400
 - 0s - loss: 0.6913 - acc: 0.5111 - val_loss: 0.6996 - val_acc: 0.4867
Epoch 12/400
 - 0s - loss: 0

Epoch 97/400
 - 0s - loss: 0.6660 - acc: 0.5778 - val_loss: 0.6801 - val_acc: 0.5133
Epoch 98/400
 - 0s - loss: 0.6696 - acc: 0.5644 - val_loss: 0.6817 - val_acc: 0.5200
Epoch 99/400
 - 0s - loss: 0.6666 - acc: 0.5711 - val_loss: 0.6832 - val_acc: 0.5067
Epoch 100/400
 - 0s - loss: 0.6676 - acc: 0.5667 - val_loss: 0.6814 - val_acc: 0.5200
Epoch 101/400
 - 0s - loss: 0.6694 - acc: 0.5689 - val_loss: 0.6844 - val_acc: 0.5133
Epoch 102/400
 - 0s - loss: 0.6707 - acc: 0.5667 - val_loss: 0.6845 - val_acc: 0.5067
Epoch 103/400
 - 0s - loss: 0.6667 - acc: 0.5689 - val_loss: 0.6845 - val_acc: 0.5200
Epoch 104/400
 - 0s - loss: 0.6661 - acc: 0.5800 - val_loss: 0.6837 - val_acc: 0.5067
Epoch 105/400
 - 0s - loss: 0.6689 - acc: 0.5733 - val_loss: 0.6815 - val_acc: 0.5200
Epoch 106/400
 - 0s - loss: 0.6679 - acc: 0.5711 - val_loss: 0.6843 - val_acc: 0.5067
Epoch 107/400
 - 0s - loss: 0.6729 - acc: 0.5689 - val_loss: 0.6855 - val_acc: 0.4933
Epoch 108/400
 - 0s - loss: 0.6706 - acc: 0.5667 - val_lo

Epoch 193/400
 - 0s - loss: 0.6646 - acc: 0.5933 - val_loss: 0.7106 - val_acc: 0.4800
Epoch 194/400
 - 0s - loss: 0.6605 - acc: 0.5778 - val_loss: 0.7116 - val_acc: 0.4733
Epoch 195/400
 - 0s - loss: 0.6668 - acc: 0.5756 - val_loss: 0.7056 - val_acc: 0.4800
Epoch 196/400
 - 0s - loss: 0.6619 - acc: 0.5911 - val_loss: 0.7077 - val_acc: 0.4867
Epoch 197/400
 - 0s - loss: 0.6594 - acc: 0.5889 - val_loss: 0.7062 - val_acc: 0.4867
Epoch 198/400
 - 0s - loss: 0.6601 - acc: 0.5844 - val_loss: 0.7078 - val_acc: 0.4867
Epoch 199/400
 - 0s - loss: 0.6605 - acc: 0.5867 - val_loss: 0.7078 - val_acc: 0.4867
Epoch 200/400
 - 0s - loss: 0.6562 - acc: 0.5911 - val_loss: 0.7097 - val_acc: 0.4800
Epoch 201/400
 - 0s - loss: 0.6518 - acc: 0.6133 - val_loss: 0.7090 - val_acc: 0.4867
Epoch 202/400
 - 0s - loss: 0.6561 - acc: 0.5978 - val_loss: 0.7081 - val_acc: 0.4867
Epoch 203/400
 - 0s - loss: 0.6568 - acc: 0.6000 - val_loss: 0.7097 - val_acc: 0.4800
Epoch 204/400
 - 0s - loss: 0.6591 - acc: 0.5756 - val

Epoch 289/400
 - 0s - loss: 0.6814 - acc: 0.5667 - val_loss: 0.6874 - val_acc: 0.5200
Epoch 290/400
 - 0s - loss: 0.6815 - acc: 0.5511 - val_loss: 0.6929 - val_acc: 0.5333
Epoch 291/400
 - 0s - loss: 0.6870 - acc: 0.5578 - val_loss: 0.6907 - val_acc: 0.5067
Epoch 292/400
 - 0s - loss: 0.6898 - acc: 0.5311 - val_loss: 0.6884 - val_acc: 0.5467
Epoch 293/400
 - 0s - loss: 0.6856 - acc: 0.5733 - val_loss: 0.6958 - val_acc: 0.5067
Epoch 294/400
 - 0s - loss: 0.6839 - acc: 0.5533 - val_loss: 0.6922 - val_acc: 0.5267
Epoch 295/400
 - 0s - loss: 0.6880 - acc: 0.5400 - val_loss: 0.6972 - val_acc: 0.5400
Epoch 296/400
 - 0s - loss: 0.6764 - acc: 0.5622 - val_loss: 0.6954 - val_acc: 0.5133
Epoch 297/400
 - 0s - loss: 0.6867 - acc: 0.5289 - val_loss: 0.6896 - val_acc: 0.5000
Epoch 298/400
 - 0s - loss: 0.6853 - acc: 0.5422 - val_loss: 0.6912 - val_acc: 0.5467
Epoch 299/400
 - 0s - loss: 0.6920 - acc: 0.5422 - val_loss: 0.6945 - val_acc: 0.5200
Epoch 300/400
 - 0s - loss: 0.6797 - acc: 0.5556 - val

Epoch 385/400
 - 0s - loss: 0.6782 - acc: 0.5533 - val_loss: 0.7067 - val_acc: 0.4800
Epoch 386/400
 - 0s - loss: 0.6821 - acc: 0.5467 - val_loss: 0.6913 - val_acc: 0.5133
Epoch 387/400
 - 0s - loss: 0.6824 - acc: 0.5511 - val_loss: 0.6952 - val_acc: 0.5133
Epoch 388/400
 - 0s - loss: 0.6855 - acc: 0.5578 - val_loss: 0.6940 - val_acc: 0.5200
Epoch 389/400
 - 0s - loss: 0.6839 - acc: 0.5400 - val_loss: 0.6858 - val_acc: 0.4933
Epoch 390/400
 - 0s - loss: 0.6801 - acc: 0.5489 - val_loss: 0.6974 - val_acc: 0.5000
Epoch 391/400
 - 0s - loss: 0.6807 - acc: 0.5511 - val_loss: 0.6937 - val_acc: 0.5067
Epoch 392/400
 - 0s - loss: 0.6791 - acc: 0.5556 - val_loss: 0.6992 - val_acc: 0.5133
Epoch 393/400
 - 0s - loss: 0.6786 - acc: 0.5622 - val_loss: 0.6808 - val_acc: 0.5333
Epoch 394/400
 - 0s - loss: 0.6788 - acc: 0.5467 - val_loss: 0.6915 - val_acc: 0.5067
Epoch 395/400
 - 0s - loss: 0.6749 - acc: 0.5578 - val_loss: 0.6924 - val_acc: 0.5067
Epoch 396/400
 - 0s - loss: 0.6880 - acc: 0.5378 - val

...

Epoch 49/400
 - 0s - loss: 0.6822 - acc: 0.5689 - val_loss: 0.6585 - val_acc: 0.5733
Epoch 50/400
 - 0s - loss: 0.6824 - acc: 0.5556 - val_loss: 0.6590 - val_acc: 0.5800
Epoch 51/400
 - 0s - loss: 0.6823 - acc: 0.5556 - val_loss: 0.6590 - val_acc: 0.5933
Epoch 52/400
 - 0s - loss: 0.6800 - acc: 0.5511 - val_loss: 0.6591 - val_acc: 0.6000
Epoch 53/400
 - 0s - loss: 0.6828 - acc: 0.5533 - val_loss: 0.6590 - val_acc: 0.5800
Epoch 54/400
 - 0s - loss: 0.6800 - acc: 0.5511 - val_loss: 0.6586 - val_acc: 0.5867
Epoch 55/400
 - 0s - loss: 0.6676 - acc: 0.5733 - val_loss: 0.6574 - val_acc: 0.5933
Epoch 56/400
 - 0s - loss: 0.6748 - acc: 0.5600 - val_loss: 0.6568 - val_acc: 0.6000
Epoch 57/400
 - 0s - loss: 0.6820 - acc: 0.5689 - val_loss: 0.6587 - val_acc: 0.6000
Epoch 58/400
 - 0s - loss: 0.6846 - acc: 0.5400 - val_loss: 0.6601 - val_acc: 0.6000
Epoch 59/400
 - 0s - loss: 0.6742 - acc: 0.5533 - val_loss: 0.6613 - val_acc: 0.6000
Epoch 60/400
 - 0s - loss: 0.6772 - acc: 0.5489 - val_loss: 0.661

Epoch 145/400
 - 0s - loss: 0.6547 - acc: 0.5800 - val_loss: 0.6687 - val_acc: 0.6400
Epoch 146/400
 - 0s - loss: 0.6526 - acc: 0.5800 - val_loss: 0.6723 - val_acc: 0.6133
Epoch 147/400
 - 0s - loss: 0.6480 - acc: 0.6133 - val_loss: 0.6699 - val_acc: 0.6333
Epoch 148/400
 - 0s - loss: 0.6488 - acc: 0.6022 - val_loss: 0.6677 - val_acc: 0.6333
Epoch 149/400
 - 0s - loss: 0.6542 - acc: 0.5978 - val_loss: 0.6695 - val_acc: 0.6333
Epoch 150/400
 - 0s - loss: 0.6517 - acc: 0.6111 - val_loss: 0.6673 - val_acc: 0.6533
Epoch 151/400
 - 0s - loss: 0.6527 - acc: 0.6089 - val_loss: 0.6656 - val_acc: 0.6600
Epoch 152/400
 - 0s - loss: 0.6561 - acc: 0.5889 - val_loss: 0.6618 - val_acc: 0.6733
Epoch 153/400
 - 0s - loss: 0.6576 - acc: 0.5867 - val_loss: 0.6595 - val_acc: 0.6800
Epoch 154/400
 - 0s - loss: 0.6497 - acc: 0.6133 - val_loss: 0.6605 - val_acc: 0.6733
Epoch 155/400
 - 0s - loss: 0.6483 - acc: 0.6111 - val_loss: 0.6685 - val_acc: 0.6333
Epoch 156/400
 - 0s - loss: 0.6449 - acc: 0.6111 - val

Epoch 241/400
 - 0s - loss: 0.6213 - acc: 0.6089 - val_loss: 0.6510 - val_acc: 0.6733
Epoch 242/400
 - 0s - loss: 0.6163 - acc: 0.6333 - val_loss: 0.6471 - val_acc: 0.6933
Epoch 243/400
 - 0s - loss: 0.6282 - acc: 0.6244 - val_loss: 0.6509 - val_acc: 0.6733
Epoch 244/400
 - 0s - loss: 0.6234 - acc: 0.6289 - val_loss: 0.6583 - val_acc: 0.6600
Epoch 245/400
 - 0s - loss: 0.6221 - acc: 0.6378 - val_loss: 0.6569 - val_acc: 0.6600
Epoch 246/400
 - 0s - loss: 0.6244 - acc: 0.6089 - val_loss: 0.6543 - val_acc: 0.6733
Epoch 247/400
 - 0s - loss: 0.6222 - acc: 0.6156 - val_loss: 0.6591 - val_acc: 0.6800
Epoch 248/400
 - 0s - loss: 0.6237 - acc: 0.6200 - val_loss: 0.6627 - val_acc: 0.6400
Epoch 249/400
 - 0s - loss: 0.6224 - acc: 0.6311 - val_loss: 0.6601 - val_acc: 0.6467
Epoch 250/400
 - 0s - loss: 0.6273 - acc: 0.6200 - val_loss: 0.6579 - val_acc: 0.6667
Epoch 251/400
 - 0s - loss: 0.6191 - acc: 0.6467 - val_loss: 0.6693 - val_acc: 0.6333
Epoch 252/400
 - 0s - loss: 0.6206 - acc: 0.6378 - val

Epoch 337/400
 - 0s - loss: 0.6371 - acc: 0.6067 - val_loss: 0.7260 - val_acc: 0.5533
Epoch 338/400
 - 0s - loss: 0.6357 - acc: 0.6333 - val_loss: 0.7301 - val_acc: 0.5467
Epoch 339/400
 - 0s - loss: 0.6419 - acc: 0.6222 - val_loss: 0.7331 - val_acc: 0.5600
Epoch 340/400
 - 0s - loss: 0.6441 - acc: 0.6222 - val_loss: 0.7311 - val_acc: 0.5467
Epoch 341/400
 - 0s - loss: 0.6454 - acc: 0.6289 - val_loss: 0.7243 - val_acc: 0.5400
Epoch 342/400
 - 0s - loss: 0.6410 - acc: 0.6200 - val_loss: 0.7250 - val_acc: 0.5467
Epoch 343/400
 - 0s - loss: 0.6302 - acc: 0.6333 - val_loss: 0.7268 - val_acc: 0.5400
Epoch 344/400
 - 0s - loss: 0.6329 - acc: 0.6289 - val_loss: 0.7295 - val_acc: 0.5267
Epoch 345/400
 - 0s - loss: 0.6273 - acc: 0.6178 - val_loss: 0.7298 - val_acc: 0.5400
Epoch 346/400
 - 0s - loss: 0.6408 - acc: 0.5978 - val_loss: 0.7308 - val_acc: 0.5533
Epoch 347/400
 - 0s - loss: 0.6391 - acc: 0.6267 - val_loss: 0.7271 - val_acc: 0.5600
Epoch 348/400
 - 0s - loss: 0.6299 - acc: 0.6333 - val

<IPython.core.display.Javascript object>

In [98]:
acc = np.asarray(res)
m, n, o = acc.shape
acc = acc.reshape(m, n)
np.mean(acc, axis=0)

array([0.60933333, 0.51533333])

In [99]:
acc

array([[0.55999999, 0.53333332],
       [0.53333333, 0.49999998],
       [0.55333333, 0.52000001],
       [0.58666667, 0.44666667],
       [0.63555556, 0.57333333],
       [0.69111111, 0.45333334],
       [0.63777778, 0.51333334],
       [0.65333334, 0.52      ],
       [0.62222222, 0.50666667],
       [0.62      , 0.58666668]])

In [103]:
plt.plot(a0, label='Training')
plt.plot(a1, label='Validation')
plt.legend()
plt.show()

In [104]:
model.save('model_tmp.h5')

In [105]:
del model

In [106]:
from keras.models import load_model
loaded_model = load_model('model_tmp.h5')

In [108]:
amat, a0, a1 = model_fit(loaded_model, batch_size, num_epochs, val_split)
res.append(amat)

Train on 450 samples, validate on 150 samples
Epoch 1/400
 - 4s - loss: 0.6663 - acc: 0.6222 - val_loss: 0.6911 - val_acc: 0.5800
Epoch 2/400
 - 0s - loss: 0.6403 - acc: 0.6333 - val_loss: 0.6862 - val_acc: 0.5733
Epoch 3/400
 - 0s - loss: 0.6379 - acc: 0.6333 - val_loss: 0.6920 - val_acc: 0.5600
Epoch 4/400
 - 0s - loss: 0.6304 - acc: 0.6511 - val_loss: 0.6902 - val_acc: 0.5600
Epoch 5/400
 - 0s - loss: 0.6307 - acc: 0.6356 - val_loss: 0.6927 - val_acc: 0.5733
Epoch 6/400
 - 0s - loss: 0.6351 - acc: 0.6533 - val_loss: 0.6924 - val_acc: 0.5800
Epoch 7/400
 - 0s - loss: 0.6319 - acc: 0.6600 - val_loss: 0.7155 - val_acc: 0.5600
Epoch 8/400
 - 0s - loss: 0.6333 - acc: 0.6444 - val_loss: 0.7183 - val_acc: 0.5800
Epoch 9/400
 - 0s - loss: 0.6305 - acc: 0.6311 - val_loss: 0.7178 - val_acc: 0.5733
Epoch 10/400
 - 0s - loss: 0.6382 - acc: 0.6244 - val_loss: 0.7234 - val_acc: 0.5867
Epoch 11/400
 - 0s - loss: 0.6400 - acc: 0.6444 - val_loss: 0.7246 - val_acc: 0.5733
Epoch 12/400
 - 0s - loss: 0

Epoch 97/400
 - 0s - loss: 0.6193 - acc: 0.6422 - val_loss: 0.7356 - val_acc: 0.5800
Epoch 98/400
 - 0s - loss: 0.6118 - acc: 0.6511 - val_loss: 0.7360 - val_acc: 0.5867
Epoch 99/400
 - 0s - loss: 0.6125 - acc: 0.6667 - val_loss: 0.7391 - val_acc: 0.5733
Epoch 100/400
 - 0s - loss: 0.6152 - acc: 0.6356 - val_loss: 0.7426 - val_acc: 0.5667
Epoch 101/400
 - 0s - loss: 0.6034 - acc: 0.6600 - val_loss: 0.7368 - val_acc: 0.5867
Epoch 102/400
 - 0s - loss: 0.6139 - acc: 0.6667 - val_loss: 0.7363 - val_acc: 0.5867
Epoch 103/400
 - 0s - loss: 0.6131 - acc: 0.6422 - val_loss: 0.7372 - val_acc: 0.5867
Epoch 104/400
 - 0s - loss: 0.6103 - acc: 0.6489 - val_loss: 0.7431 - val_acc: 0.5867
Epoch 105/400
 - 0s - loss: 0.6090 - acc: 0.6578 - val_loss: 0.7535 - val_acc: 0.5800
Epoch 106/400
 - 0s - loss: 0.6154 - acc: 0.6467 - val_loss: 0.7498 - val_acc: 0.5800
Epoch 107/400
 - 0s - loss: 0.6100 - acc: 0.6556 - val_loss: 0.7492 - val_acc: 0.5667
Epoch 108/400
 - 0s - loss: 0.6037 - acc: 0.6556 - val_lo

Epoch 193/400
 - 0s - loss: 0.5918 - acc: 0.6756 - val_loss: 0.7662 - val_acc: 0.5600
Epoch 194/400
 - 0s - loss: 0.5918 - acc: 0.6667 - val_loss: 0.7612 - val_acc: 0.5600
Epoch 195/400
 - 0s - loss: 0.5825 - acc: 0.6689 - val_loss: 0.7614 - val_acc: 0.5733
Epoch 196/400
 - 0s - loss: 0.5974 - acc: 0.6667 - val_loss: 0.7609 - val_acc: 0.5667
Epoch 197/400
 - 0s - loss: 0.5943 - acc: 0.6644 - val_loss: 0.7656 - val_acc: 0.5667
Epoch 198/400
 - 0s - loss: 0.5919 - acc: 0.6778 - val_loss: 0.7699 - val_acc: 0.5733
Epoch 199/400
 - 0s - loss: 0.5927 - acc: 0.6600 - val_loss: 0.7584 - val_acc: 0.5667
Epoch 200/400
 - 0s - loss: 0.5700 - acc: 0.6911 - val_loss: 0.7573 - val_acc: 0.5667
Epoch 201/400
 - 0s - loss: 0.5870 - acc: 0.6822 - val_loss: 0.7500 - val_acc: 0.5800
Epoch 202/400
 - 0s - loss: 0.5984 - acc: 0.6644 - val_loss: 0.7436 - val_acc: 0.5733
Epoch 203/400
 - 0s - loss: 0.5796 - acc: 0.6822 - val_loss: 0.7463 - val_acc: 0.5800
Epoch 204/400
 - 0s - loss: 0.5768 - acc: 0.6756 - val

Epoch 289/400
 - 0s - loss: 0.6093 - acc: 0.6578 - val_loss: 0.7713 - val_acc: 0.5867
Epoch 290/400
 - 0s - loss: 0.5856 - acc: 0.6756 - val_loss: 0.7746 - val_acc: 0.5933
Epoch 291/400
 - 0s - loss: 0.5875 - acc: 0.6600 - val_loss: 0.7745 - val_acc: 0.5933
Epoch 292/400
 - 0s - loss: 0.5944 - acc: 0.6689 - val_loss: 0.7675 - val_acc: 0.6133
Epoch 293/400
 - 0s - loss: 0.5966 - acc: 0.6533 - val_loss: 0.7705 - val_acc: 0.6067
Epoch 294/400
 - 0s - loss: 0.5862 - acc: 0.6667 - val_loss: 0.7712 - val_acc: 0.5867
Epoch 295/400
 - 0s - loss: 0.5950 - acc: 0.6600 - val_loss: 0.7651 - val_acc: 0.6067
Epoch 296/400
 - 0s - loss: 0.5888 - acc: 0.6600 - val_loss: 0.7663 - val_acc: 0.6000
Epoch 297/400
 - 0s - loss: 0.5914 - acc: 0.6711 - val_loss: 0.7665 - val_acc: 0.6000
Epoch 298/400
 - 0s - loss: 0.5671 - acc: 0.6956 - val_loss: 0.7728 - val_acc: 0.6067
Epoch 299/400
 - 0s - loss: 0.5911 - acc: 0.6600 - val_loss: 0.7761 - val_acc: 0.6067
Epoch 300/400
 - 0s - loss: 0.5912 - acc: 0.6622 - val

Epoch 385/400
 - 0s - loss: 0.5653 - acc: 0.6978 - val_loss: 0.8071 - val_acc: 0.6333
Epoch 386/400
 - 0s - loss: 0.5536 - acc: 0.6956 - val_loss: 0.7976 - val_acc: 0.6467
Epoch 387/400
 - 0s - loss: 0.5659 - acc: 0.7067 - val_loss: 0.7928 - val_acc: 0.6333
Epoch 388/400
 - 0s - loss: 0.5606 - acc: 0.6978 - val_loss: 0.7794 - val_acc: 0.6533
Epoch 389/400
 - 0s - loss: 0.5629 - acc: 0.6933 - val_loss: 0.7686 - val_acc: 0.6533
Epoch 390/400
 - 0s - loss: 0.5717 - acc: 0.6733 - val_loss: 0.7604 - val_acc: 0.6467
Epoch 391/400
 - 0s - loss: 0.5581 - acc: 0.6822 - val_loss: 0.7664 - val_acc: 0.6333
Epoch 392/400
 - 0s - loss: 0.5664 - acc: 0.6822 - val_loss: 0.7604 - val_acc: 0.6400
Epoch 393/400
 - 0s - loss: 0.5522 - acc: 0.6933 - val_loss: 0.7700 - val_acc: 0.6467
Epoch 394/400
 - 0s - loss: 0.5676 - acc: 0.6800 - val_loss: 0.7589 - val_acc: 0.6533
Epoch 395/400
 - 0s - loss: 0.6052 - acc: 0.6533 - val_loss: 0.7806 - val_acc: 0.6000
Epoch 396/400
 - 0s - loss: 0.5987 - acc: 0.6400 - val

In [109]:
res

[[[0.5599999931123522], [0.5333333214124044]],
 [[0.5333333333333333], [0.49999998410542806]],
 [[0.5533333261807759], [0.5200000067551931]],
 [[0.5866666674613953], [0.4466666678587596]],
 [[0.6355555574099223], [0.5733333269755045]],
 [[0.6911111076672872], [0.4533333381017049]],
 [[0.6377777788374159], [0.5133333384990693]],
 [[0.6533333354526096], [0.5200000047683716]],
 [[0.6222222208976745], [0.5066666722297668]],
 [[0.6200000034438239], [0.5866666754086812]],
 [[0.6733333283000522], [0.6133333444595337]]]